**Домашнее задание**

Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
1. Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей
2. Построить случайный лес, вывести важность признаков
3. Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
4. Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии
5. Показать, что использование ансамблей моделей действительно улучшает качество (стекинг vs другие модели сравнивать на hold-out)
6. В качестве решения: Jupyter notebook с кодом, комментариями и графиками.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error

In [2]:
data = pd.read_csv("train.csv")
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

Кол-во признаков очень большое, отберем несколько из них, чтобы можно было построить модели, требуемые в задании, за разумное время.

In [4]:
data = data[["MSZoning","LotFrontage","LotArea","Street","OverallQual","OverallCond","YearBuilt","MasVnrArea",
      "CentralAir","Fireplaces","GarageType","SaleType","SaleCondition","SalePrice"]]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSZoning       1460 non-null   object 
 1   LotFrontage    1201 non-null   float64
 2   LotArea        1460 non-null   int64  
 3   Street         1460 non-null   object 
 4   OverallQual    1460 non-null   int64  
 5   OverallCond    1460 non-null   int64  
 6   YearBuilt      1460 non-null   int64  
 7   MasVnrArea     1452 non-null   float64
 8   CentralAir     1460 non-null   object 
 9   Fireplaces     1460 non-null   int64  
 10  GarageType     1379 non-null   object 
 11  SaleType       1460 non-null   object 
 12  SaleCondition  1460 non-null   object 
 13  SalePrice      1460 non-null   int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 159.8+ KB


Заполним пропуски значений

LotFrontage - непрерывная величина, заполним по медиане, т.к. она более устойчива к выбросам.

In [5]:
lf_median = data["LotFrontage"].median()
data["LotFrontage"].fillna(lf_median, inplace=True)

Аналогично с MasVnrArea.

In [6]:
mv_median = data["MasVnrArea"].median()
data["MasVnrArea"].fillna(mv_median, inplace=True)

GarageType - категориальная переменная, посмотрим моду.

In [7]:
data["GarageType"].value_counts()

Attchd     870
Detchd     387
BuiltIn     88
Basment     19
CarPort      9
2Types       6
Name: GarageType, dtype: int64

Мода - явная, заполним по моде.

In [8]:
data["GarageType"].fillna("Attchd", inplace=True)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSZoning       1460 non-null   object 
 1   LotFrontage    1460 non-null   float64
 2   LotArea        1460 non-null   int64  
 3   Street         1460 non-null   object 
 4   OverallQual    1460 non-null   int64  
 5   OverallCond    1460 non-null   int64  
 6   YearBuilt      1460 non-null   int64  
 7   MasVnrArea     1460 non-null   float64
 8   CentralAir     1460 non-null   object 
 9   Fireplaces     1460 non-null   int64  
 10  GarageType     1460 non-null   object 
 11  SaleType       1460 non-null   object 
 12  SaleCondition  1460 non-null   object 
 13  SalePrice      1460 non-null   int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 159.8+ KB


Street и CentralAir имеют всего 2 значения, закодируем их через LabelEncoder.

In [10]:
def lab_enc(sr):
    lab = LabelEncoder()
    lab_f = lab.fit_transform(sr)
    return pd.Series(lab_f, name=sr.name)

data["Street"] = lab_enc(data["Street"])
data["CentralAir"] = lab_enc(data["CentralAir"])

Проведем нормализацию года постройки, вычитая из максимального каждое из значений.

In [11]:
yb_max = data["YearBuilt"].max()
data["Age"] = data["YearBuilt"].apply(lambda x: yb_max - x)

Признаки: MSZoning, GarageType, SaleType, SaleCondition - категориальные и несравнимые. Закодируем их через OneHotEncoder.

In [12]:
def ohe_enc(df, prefix = 'col'):
    ohe = OneHotEncoder()
    ohe_f = ohe.fit_transform(df).toarray()
    return pd.DataFrame(ohe_f, index=df.index, columns=[str(prefix) + str(x) for x in ohe.categories_[0]])

data = data.join(ohe_enc(data[["MSZoning"]], "MSZoning_"))
data = data.join(ohe_enc(data[["GarageType"]], "GarageType_"))
data = data.join(ohe_enc(data[["SaleType"]], "SaleType_"))
data = data.join(ohe_enc(data[["SaleCondition"]], "SaleCondition_"))
data

,MSZoning,LotFrontage,LotArea,Street,OverallQual,OverallCond,YearBuilt,MasVnrArea,CentralAir,Fireplaces,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,RL,65.0,8450,1,7,5,2003,196.0,1,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,RL,80.0,9600,1,6,8,1976,0.0,1,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,RL,68.0,11250,1,7,5,2001,162.0,1,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,RL,60.0,9550,1,7,5,1915,0.0,1,1,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,RL,84.0,14260,1,8,5,2000,350.0,1,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,62.0,7917,1,6,5,1999,0.0,1,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1456,RL,85.0,13175,1,6,6,1978,119.0,1,2,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1457,RL,66.0,9042,1,7,9,1941,0.0,1,2,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1458,RL,68.0,9717,1,5,6,1950,0.0,1,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


Оставшиеся признаки пропустим через Scaler.

In [13]:
def scaler(sr):
    return StandardScaler().fit_transform(sr)

xr = ["LotFrontage","LotArea","OverallQual","OverallCond","Age","MasVnrArea","Fireplaces"]
data[xr] = scaler(data[xr])
data[xr]

,LotFrontage,LotArea,OverallQual,OverallCond,Age,MasVnrArea,Fireplaces
0,-0.220875,-0.207142,0.651479,-0.517200,-1.050994,0.514104,-0.951226
1,0.460320,-0.091886,-0.071836,2.179628,-0.156734,-0.570750,0.600495
2,-0.084636,0.073480,0.651479,-0.517200,-0.984752,0.325915,0.600495
3,-0.447940,-0.096897,0.651479,-0.517200,1.863632,-0.570750,0.600495
4,0.641972,0.375148,1.374795,-0.517200,-0.951632,1.366489,0.600495
...,...,...,...,...,...,...,...
1455,-0.357114,-0.260560,-0.071836,-0.517200,-0.918511,-0.570750,0.600495
1456,0.687385,0.266407,-0.071836,0.381743,-0.222975,0.087911,2.152216
1457,-0.175462,-0.147810,0.651479,3.078570,1.002492,-0.570750,2.152216
1458,-0.084636,-0.080160,-0.795151,0.381743,0.704406,-0.570750,-0.951226


Удаляем более ненужные столбцы и получаем датафрейм, на котором будем производить обучение.

In [14]:
data.drop(columns=["YearBuilt", "MSZoning", "GarageType", "SaleType", "SaleCondition"], inplace=True)
data

,LotFrontage,LotArea,Street,OverallQual,OverallCond,MasVnrArea,CentralAir,Fireplaces,SalePrice,Age,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-0.220875,-0.207142,1,0.651479,-0.517200,0.514104,1,-0.951226,208500,-1.050994,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.460320,-0.091886,1,-0.071836,2.179628,-0.570750,1,0.600495,181500,-0.156734,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.084636,0.073480,1,0.651479,-0.517200,0.325915,1,0.600495,223500,-0.984752,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.447940,-0.096897,1,0.651479,-0.517200,-0.570750,1,0.600495,140000,1.863632,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.641972,0.375148,1,1.374795,-0.517200,1.366489,1,0.600495,250000,-0.951632,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-0.357114,-0.260560,1,-0.071836,-0.517200,-0.570750,1,0.600495,175000,-0.918511,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1456,0.687385,0.266407,1,-0.071836,0.381743,0.087911,1,2.152216,210000,-0.222975,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1457,-0.175462,-0.147810,1,0.651479,3.078570,-0.570750,1,2.152216,266500,1.002492,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1458,-0.084636,-0.080160,1,-0.795151,0.381743,-0.570750,1,-0.951226,142125,0.704406,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


Делим выборку на 2 части: тренировочный и hold-out датасет. Затем делим тренировочный датасет на тенировочную и тестовую выборки.

In [15]:
y = data["SalePrice"]
X = data.drop(columns=["SalePrice"])

X_base, X_holdout, y_base, y_holdout = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_base, y_base, test_size=0.3, random_state=42)

Строим случайный лес.

In [16]:
rf = RandomForestRegressor(n_estimators=25, max_depth=10, min_samples_leaf=25, max_features=0.5, n_jobs=-1)
rf.fit(X_train, y_train);

Выведем важность признаков.

In [17]:
imp = pd.Series(rf.feature_importances_, index=X_train.columns)
sv = imp.sort_values(ascending=False)
sv[sv != 0]

OverallQual             0.540706
Age                     0.206608
LotArea                 0.110673
Fireplaces              0.071331
MasVnrArea              0.037112
LotFrontage             0.012309
GarageType_Attchd       0.005568
OverallCond             0.003223
SaleCondition_Normal    0.003003
MSZoning_RM             0.002955
CentralAir              0.002777
MSZoning_RL             0.002139
GarageType_Detchd       0.001596
dtype: float64

Выполним стекинг.

In [18]:
estimators = [
    ("Linear", LinearRegression()),
    ("Polynomial", make_pipeline(PolynomialFeatures(2, interaction_only=False, include_bias=False), LinearRegression())),
    ("Support Vectors", SVR(kernel="poly", degree=9))
]

sr = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression()
)

sr.fit(X_train, y_train);

Выполним блендинг.

In [19]:
results = []
models = []
for est in estimators:
    model = est[1]
    model.fit(X_train, y_train)
    models.append([est[0], model])
    results.append(model.predict(X_test))
results = np.array(results).T

bl = LinearRegression()
bl.fit(results, y_test);

Оценим качество моделей на hold-out датасете.

In [20]:
def score(m, x, y, cv=10):
    scores = cross_val_score(m, x, y, cv=cv)
    lst = [score for score in scores if score >= 0] # drop anomaly values
    return np.mean(lst)

results = []
scores = []
for model in models:
    scores.append([model[0], score(model[1], X_holdout, y_holdout)])
    results.append(model[1].predict(X_holdout)) # predict for the final table
results = np.array(results).T

scores.append(["Random Forest", score(rf, X_holdout, y_holdout)])
scores.append(["Stacking", score(sr, X_holdout, y_holdout)])
scores.append(["Blanding", score(bl, results, y_holdout)])
pd.DataFrame(scores, columns=["Алгоритм", "Качество"])

,Алгоритм,Качество
0,Linear,0.714530
1,Polynomial,0.346062
2,Support Vectors,0.072220
3,Random Forest,0.682580
4,Stacking,0.720463
5,Blanding,0.736871
